In [ ]:
import os, sys
import pandas as pd
from sklearn.manifold import MDS
from sklearn.cluster import KMeans

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)


from  techminer import *

# import_scopus_file(scopus_file='raw-documents.csv',)
# import_references(scopus_file="raw-references.csv")




directory = "/workspaces/techminer-api/data/"
matrix = coupling_by_column_matrix(
    column='author_keywords',
    min_occ=6, 
    top_n=100,
    directory=directory,
)
coupling_net =  coupling_network(matrix, directory=directory)
# network_plot(coupling_net, max_labels=10,k=0.001, iterations=100)
network_communities(coupling_net)

In [ ]:
import pandas as pd

documents = pd.read_csv("documents.csv")
documents = documents.sort_values("local_citations", ascending=False)
for row in documents.head(20).iterrows():
    print(row[1]["local_citations"], row[1]["document_title"])
    

In [ ]:
import pandas as pd

references = pd.read_csv("references.csv")
references = references.sort_values("local_citations", ascending=False)
for row in references.head(20).iterrows():
    print(row[1]["local_citations"], row[1]["document_title"])

In [ ]:
import pandas as pd

pd.read_csv("references.csv").authors.str.split(";")

In [ ]:
from techminer import *
directory = "/workspaces/techminer-api/data/"
collaboration_indicators("countries", directory=directory).head()

In [ ]:
import pandas as pd

m = pd.DataFrame(
    {
    'a': [1, 1, 1, 2, 2, 2],
    'b': ['A', 'B', 'D', 'A', 'B', 'C'],
    'v': [1, 2, 3, 4, 5, 6]
    }
)
m.pivot(index='a', columns='b', values='v')